In [1]:

import tganalysis
import os
import emoji
import pandas as pd

paths = ['C:/Users/mk/Downloads/Telegram Desktop/ChatExport_2023-12-30_']

filename = os.path.join('data', paths[0], 'result.json')


In [2]:

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
emb = NewsEmbedding()
segmenter = Segmenter()
morph_vocab = MorphVocab()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)


In [3]:
tg = tganalysis.tg_json_analysis(filename)
data = tg.df
data

{521643: msg(id=521643, type='message', date='2023-10-01T00:09:46', from_name='Ксения Щербак', from_id='user239502090', reply_to_message_id=521634, text='У меня раньше было такое, что я не могла каждый раз его юзать, был перепрот( все разваливалось и прямилось). А сейчас у меня всегда ливином либо он, либо шелк dnc и все супер) видимо раньше были волосы недостаточно увлажнены', words=None, year=2023, month=10, day=1, week=39, weekday=7, hour=0, td_idf=None),
 521644: msg(id=521644, type='message', date='2023-10-01T00:12:31', from_name='Ксения Щербак', from_id='user239502090', reply_to_message_id=521636, text='Я задумалась о смене состава после ваших слов о трех видах протеина) потому что я не вижу там их в таком количестве \nИ в карте товара состав отличается от того, что на баночке моей', words=None, year=2023, month=10, day=1, week=39, weekday=7, hour=0, td_idf=None),
 521645: msg(id=521645, type='message', date='2023-10-01T00:15:01', from_name='Mariya Gudova', from_id='user349508138

In [4]:
# data[522845].text
# text = data[522835].text


docs = list()

for dp in data:
    text = data[dp].text
    text = emoji.replace_emoji(text, lambda a,_:f".{a}. ")
    text = text.replace(') ', "). ")
    text = text.replace('\n', "\n. ")
    text = '. '.join([a.capitalize() for a in text.split('. ')])
    try:
        doc = Doc(text)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        doc.parse_syntax(syntax_parser)
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
        docs.append({'id': dp, 'from_name': data[dp].from_name, 'doc': doc})
    except:
        pass
    




In [5]:
# text = 'Мне алоэ плоховато распутывает и не очень делал волосы скользкими, хотя обычно у меня такой проблемы нет, в итоге получался очень большой расход'
# text = emoji.replace_emoji(text, lambda a,_:f".{a}. ")
# text = text.replace(') ', "). ")
# text = text.replace('\n', "\n. ")
# text = '. '.join([a.capitalize() for a in text.split('. ')])
# doc = Doc(text)
# doc.segment(segmenter)
# doc.tag_morph(morph_tagger)
# doc.parse_syntax(syntax_parser)
# for token in doc.tokens:
#     token.lemmatize(morph_vocab)
# doc.tokens

In [6]:
roots_all = []
for doc_index, doc in enumerate(docs):
    for sent_index, sent in enumerate(doc['doc'].sents):
        roots = dict()
        for token in sent.tokens:
            if (token.rel == 'root') and (token.pos == 'VERB'):
                if not token.id in roots:
                    roots[token.id] = {'doc_index': doc_index, 'sent_index': sent_index}
                roots[token.id]['root'] = token
            if token.rel == 'obj':
                try:
                    root_token = sent.tokens[int(token.head_id[2:])-1]
                except:
                    root_token = sent.tokens[int(token.head_id[3:])-1]
                if not (root_token.rel == 'root' and root_token.pos == 'VERB'):
                    continue
                if not token.head_id in roots:
                    roots[token.head_id] = {'doc_index': doc_index, 'sent_index': sent_index}
                roots[token.head_id]['obj'] = token
            if token.rel == 'nsubj':
                try:
                    root_token = sent.tokens[int(token.head_id[2:])-1]
                except:
                    root_token = sent.tokens[int(token.head_id[3:])-1]
                if not (root_token.rel == 'root' and root_token.pos == 'VERB'):
                    continue
                if not token.head_id in roots:
                    roots[token.head_id] = {'doc_index': doc_index, 'sent_index': sent_index}
                roots[token.head_id]['nsubj'] = token
            if token.rel == 'advmod':
                try:
                    root_token = sent.tokens[int(token.head_id[2:])-1]
                except:
                    root_token = sent.tokens[int(token.head_id[3:])-1]
                if not (root_token.rel == 'root' and root_token.pos == 'VERB'):
                    continue
                if not token.head_id in roots:
                    roots[token.head_id] = {'doc_index': doc_index, 'sent_index': sent_index}
                roots[token.head_id]['advmod'] = token
        if roots:
            roots_all.append(roots)                
        # print(roots)
        # break


In [7]:
df = list()
for roots in roots_all:
    for root_id, root in roots.items():
        from_name = docs[root['doc_index']]['from_name']
        msg_id = docs[root['doc_index']]['id']
        d = {'from_name': from_name, 'msg_id': msg_id}
        try:
            d['advmod'] = root['advmod'].lemma
        except:
            pass
        try:
            d['root'] = root['root'].lemma
        except:
            pass
        try:
            d['nsubj'] = root['nsubj'].lemma
        except:
            pass
        try:
            d['obj'] = root['obj'].lemma
        except:
            pass
        df.append(d)
    # break
# doc[roots]

df = pd.DataFrame(df)

# df


In [8]:
data[544380]

msg(id=544380, type='message', date='2023-11-09T01:07:02', from_name='Daria', from_id='user940684088', reply_to_message_id=544249, text='Мне алоэ плоховато распутывает и не очень делал волосы скользкими, хотя обычно у меня такой проблемы нет, в итоге получался очень большой расход', words=None, year=2023, month=11, day=9, week=45, weekday=4, hour=1, td_idf=None)

In [9]:

df.fillna('').groupby(['from_name', 'nsubj', 'advmod', 'root', 'obj']).count().reset_index()[['msg_id', 'nsubj', 'advmod', 'root', 'obj']].groupby(['nsubj', 'advmod', 'root', 'obj']).sum().to_clipboard()

In [240]:
df.query('root == "алоэ"').sort_values('obj')

,from_name,msg_id,advmod,root,nsubj,obj
21626,Daria,544380,NaN,алоэ,NaN,NaN


In [226]:
df.pivot_table(index='from_name', columns='root', values='msg_id', aggfunc='count')

root,аааа,абсорбировать,абьюзита,аватарка,ага,агафья,адаптироваться,адвент-календарь,аж,айсель,...,юзать,юзер,юзнуть,я-,я-то,являться,ядреный,як,ярость,ясный
from_name,,,,,,,,,,,,,,,,,,,,,
,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
🕊️ᴬᴸᴮᴵᴺᴬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
🟢Екатерина Скоблинская🟢,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
🥭,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [215]:

df.groupby(['from_name', 'root']).count().sort_values('from_name').tail(30)

msg_id  advmod  nsubj  obj
from_name root                                  
🦊 Олеся   задуматься       1       1      1    0
          оказаться        1       0      1    0
          читать           2       1      2    2
          быть             5       2      4    0
          везти            1       0      0    0
          верить           1       0      0    0
          взять            1       0      0    0
          видеть           2       1      1    1
          висеть           1       1      0    0
          влиять           1       1      1    0
          вымываться       1       1      1    0
          говорить         4       2      3    0
          давать           2       1      0    0
          делать           1       1      1    1
          открывать        1       0      0    1
          думать           1       0      1    1
          зайти            1       1      0    0
          заметить         1       0      1    1
          знать            2       1      1    0
          идти             1       0      0    0
          кокос            1       1      0    0
          купить           3       0      1    2
          лежать           1       1      0    0
          мочь             5       2      3    1
          наблюдать        1       0      0    0
          находить         1       1      0    1
          начинаться       1       0      1    0
          нет              1       0      1    0
          задать           1       0      1    1
          шлаки            1       1      1    0

In [214]:
df.to_clipboard()
#.query('root == "стоить"')
#.groupby('root').count().sort_values('from_name').tail(30)

In [182]:
docs[-1]

{'id': 569203,
 'from_name': 'Red Melon',
 'doc': Doc(text='Да, примерно тоже самое). Взаимозаменимы\n. Кокос..., tokens=[...], sents=[...])}

In [145]:
# text = 'Открытки “С днем 8 Марта” в СССР начали выпускать лишь с 1952 года.\n\nКаждый год выпускалось от 10 до 80 видов открыток. Над их созданием трудились лучшие художники, выпускались специальные тематические серии и целые коллекционные альбомы.'
text = 'Женщина сильно ругалась.'

doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
for token in doc.tokens:
    token.lemmatize(morph_vocab)

doc.tokens

[DocToken(stop=7, text='Женщина', id='1_1', head_id='1_3', rel='nsubj', pos='NOUN', feats=<Anim,Nom,Fem,Sing>, lemma='женщина'),
 DocToken(start=8, stop=14, text='сильно', id='1_2', head_id='1_3', rel='advmod', pos='ADV', feats=<Pos>, lemma='сильно'),
 DocToken(start=15, stop=23, text='ругалась', id='1_3', head_id='1_0', rel='root', pos='VERB', feats=<Imp,Fem,Ind,Sing,Past,Fin,Act>, lemma='ругаться'),
 DocToken(start=23, stop=24, text='.', id='1_4', head_id='1_3', rel='punct', pos='PUNCT', lemma='.')]

[DocToken(stop=8, text='Согласна', id='1_1', head_id='1_0', rel='root', pos='ADJ', feats=<Pos,Fem,Sing,Short>, lemma='согласный'),
 DocToken(start=8, stop=9, text=')', id='1_2', head_id='1_1', rel='punct', pos='PUNCT', lemma=')'),
 DocToken(start=10, stop=11, text='я', id='1_3', head_id='1_4', rel='nsubj', pos='PRON', feats=<Nom,Sing,1>, lemma='я'),
 DocToken(start=12, stop=20, text='стараюсь', id='1_4', head_id='1_0', rel='root', pos='VERB', feats=<Imp,Ind,Sing,1,Pres,Fin,Mid>, lemma='стараться'),
 DocToken(start=21, stop=28, text='держать', id='1_5', head_id='1_4', rel='xcomp', pos='VERB', feats=<Imp,Inf,Act>, lemma='держать'),
 DocToken(start=29, stop=33, text='себя', id='1_6', head_id='1_5', rel='obj', pos='PRON', feats=<Acc>, lemma='себя'),
 DocToken(start=34, stop=35, text='в', id='1_7', head_id='1_8', rel='case', pos='ADP', lemma='в'),
 DocToken(start=36, stop=41, text='руках', id='1_8', head_id='1_5', rel='obl', pos='NOUN', feats=<Inan,Loc,Fem,Plur>, lemma='рука'),
 DocToken(st